# Trabajo Práctico 2 - Organización de Datos
## Competencia de Machine Learning
### Facultad de Ingeniería de la Universidad de Buenos Aires
### 95-58: Organización de Datos - 2do Cuat. 2018

#### Integrantes: Gonzalo Diz,  Ariel Windey, Gabriel Robles y Matías El Dócil




#### Objetivo
Determinar, para cada usuario presentado, cuál es la probabilidad de que ese
usuario realice una conversión en Trocafone en un periodo determinado.

#### Fuentes
El archivo "events_up_to_01062018.csv" contiene en el mismo formato utilizado en el TP1
información de eventos realizado en la plataforma para un conjunto de usuarios hasta el
31/05/2018.

Por otro lado el archivo "labels_training_set.csv" indica para un subconjunto de los
usuarios incluidos en el set de eventos "events_up_to_01062018.csv" si los mismos
realizaron una conversión (columna label = 1) o no (columna label = 0) desde el 01/06/2018
hasta el 15/06/2018.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn as skl

%matplotlib inline

pd.set_option('display.max_columns', 100)

In [2]:
# Carga del set de datos de eventos
eventos = pd.read_csv('../dataset/events_up_to_01062018.csv', low_memory=False)
# Carga del set de datos de labels
labels = pd.read_csv('../dataset/labels_training_set.csv', low_memory=False)


In [3]:
eventos.shape

(2341681, 23)

In [4]:
eventos.event.value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [5]:
eventos.sample(5)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
2171383,2018-05-22 01:48:34,visited site,1fb7a7cd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Direct,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 7.1.1,Chrome Mobile 66.0
1539183,2018-05-24 17:14:22,viewed product,90627c8f,NaN,6846.0,iPhone 6S,Muito Bom,16GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2241574,2018-05-27 17:08:32,visited site,c8e25442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Referral,Returning,Volta Redonda,Rio de Janeiro,Brazil,Computer,1360x768,Windows 7,Chrome 66.0
2080021,2018-03-30 14:07:36,brand listing,e1443dd4,NaN,NaN,NaN,NaN,NaN,NaN,"6819,3191,6805,8555,3180,2718,6413,2784,2703,2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1795298,2018-05-26 21:17:47,generic listing,6be8c99a,NaN,NaN,NaN,NaN,NaN,NaN,"2820,6706,2750,6720,6649,7252,7224,6663,12604,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Formateo los eventos
eventos['timestamp'] = pd.to_datetime(eventos['timestamp'])

In [7]:
# Obtenemos un frame con solo los usuarios donde iremos mergeando los features
persons = eventos.person.to_frame().drop_duplicates()

### Región geográfica
Agregamos la región geográfica al igual que en el TP 1, para saber desde donde acceden más los distintos usuarios.

In [8]:
# Categorizamos las regiones geograficamente
eventos['geo_region'] = 0

eventos.loc[
    (eventos['region'] == 'Sao Paulo') |
    (eventos['region'] == 'Rio de Janeiro') |
    (eventos['region'] == 'Minas Gerais') |
    (eventos['region'] == 'Espirito Santo'), 
    'geo_region'
] = 'southeast'

eventos.loc[
    (eventos['region'] == 'Parana') |
    (eventos['region'] == 'Rio Grande do Sul') |
    (eventos['region'] == 'Santa Catarina'), 
    'geo_region'
] = 'south'

eventos.loc[
    (eventos['region'] == 'Federal District') |
    (eventos['region'] == 'Goias') |
    (eventos['region'] == 'Mato Grosso do Sul') |
    (eventos['region'] == 'Mato Grosso'), 
    'geo_region'
] = 'center west'

eventos.loc[
    (eventos['region'] == 'Bahia') |
    (eventos['region'] == 'Pernambuco') |
    (eventos['region'] == 'Ceara') |
    (eventos['region'] == 'Maranhao') |
    (eventos['region'] == 'Rio Grande do Norte') |
    (eventos['region'] == 'Paraíba') |
    (eventos['region'] == 'Piaui') |
    (eventos['region'] == 'Alagoas') |
    (eventos['region'] == 'Sergipe'), 
    'geo_region'
] = 'northeast'

eventos.loc[
    (eventos['region'] == 'Para') |
    (eventos['region'] == 'Amazonas') |
    (eventos['region'] == 'Tocantins') |
    (eventos['region'] == 'Amapa') |
    (eventos['region'] == 'Rondonia') |
    (eventos['region'] == 'Acre') |
    (eventos['region'] == 'Roraima'), 
    'geo_region'
] = 'north'

In [9]:
eventos.geo_region.value_counts()

0              2172865
southeast        97323
northeast        40903
south            14242
center west       9249
north             7099
Name: geo_region, dtype: int64

Creamos columnas booleanas sobre las regiones geograficas para luego agrupar por usuarios.

In [10]:
eventos['cant_accesos_southeast'] = (eventos.geo_region == 'southeast').astype(int)
eventos['cant_accesos_northeast'] = (eventos.geo_region == 'northeast').astype(int)
eventos['cant_accesos_south'] = (eventos.geo_region == 'south').astype(int)
eventos['cant_accesos_center_west'] = (eventos.geo_region == 'center west').astype(int)
eventos['cant_accesos_north'] = (eventos.geo_region == 'north').astype(int)

In [11]:
personas_y_accesos_por_region = eventos.groupby(['person']).agg({
    'cant_accesos_southeast': 'sum', 
    'cant_accesos_northeast': 'sum', 
    'cant_accesos_south': 'sum', 
    'cant_accesos_center_west': 'sum', 
    'cant_accesos_north': 'sum'
}).reset_index()
personas_y_accesos_por_region.head(5)

,person,cant_accesos_north,cant_accesos_center_west,cant_accesos_southeast,cant_accesos_south,cant_accesos_northeast
0,0008ed71,0,0,0,0,0
1,00091926,0,0,0,34,0
2,00091a7a,0,0,1,0,0
3,000ba417,0,0,6,0,0
4,000c79fe,0,0,1,0,0


Agregamos este nuevo feature a nuestro dataframe de features.

In [13]:
user_features = persons.merge(personas_y_accesos_por_region, how='inner', on='person')
user_features.head(5)

,person,cant_accesos_north,cant_accesos_center_west,cant_accesos_southeast,cant_accesos_south,cant_accesos_northeast
0,4886f805,0,0,1,0,0
1,ad93850f,0,0,5,0,0
2,0297fc1e,0,0,95,0,0
3,2d681dd8,0,0,2,0,0
4,cccea85e,0,0,22,0,0


In [14]:
user_features.to_csv('features_ariel.csv')